
# 🏨 LimaSmart - Optimización de Tarifas Dinámicas con Algoritmos Genéticos

Este notebook aplica mecanismos de selección, cruce y mutación sobre un conjunto de precios binarios para maximizar el ingreso estimado de un hotel urbano.


## Paso 1: Cargar y visualizar el dataset

In [ ]:

import pandas as pd
import random
import matplotlib.pyplot as plt

# Cargar el dataset
df = pd.read_csv("lima_smart_tarifas.csv")
df = df.sort_values(by="Fitness", ascending=False).reset_index(drop=True)
df


## Paso 2: Implementar mecanismos de selección

In [ ]:

def seleccion_ruleta(df, n=4):
    total_fit = df['Fitness'].sum()
    seleccionados = []
    for _ in range(n):
        pick = random.uniform(0, total_fit)
        current = 0
        for _, row in df.iterrows():
            current += row['Fitness']
            if current >= pick:
                seleccionados.append(row['Cromosoma'])
                break
    return seleccionados

def seleccion_torneo(df, n=4, k=3):
    seleccionados = []
    for _ in range(n):
        grupo = df.sample(k)
        ganador = grupo.loc[grupo['Fitness'].idxmax()]
        seleccionados.append(ganador['Cromosoma'])
    return seleccionados

def seleccion_elitismo(df, n=4):
    return df.head(n)['Cromosoma'].tolist()

def seleccion_ranking(df, n=4):
    df_sorted = df.sort_values(by='Fitness').reset_index(drop=True)
    df_sorted['Rank'] = df_sorted.index + 1
    total_ranks = df_sorted['Rank'].sum()
    seleccionados = []
    for _ in range(n):
        pick = random.uniform(0, total_ranks)
        current = 0
        for _, row in df_sorted.iterrows():
            current += row['Rank']
            if current >= pick:
                seleccionados.append(row['Cromosoma'])
                break
    return seleccionados

def seleccion_estocastica(df, n=4):
    df = df.copy()
    df['Probabilidad'] = df['Fitness'] / df['Fitness'].sum()
    df['Acumulada'] = df['Probabilidad'].cumsum()
    start = random.uniform(0, 1/n)
    puntos = [start + i/n for i in range(n)]
    seleccionados = []
    for punto in puntos:
        for _, row in df.iterrows():
            if punto <= row['Acumulada']:
                seleccionados.append(row['Cromosoma'])
                break
    return seleccionados


## Paso 3: Aplicar cada mecanismo de selección

In [ ]:

ruleta = seleccion_ruleta(df)
torneo = seleccion_torneo(df)
elitismo = seleccion_elitismo(df)
ranking = seleccion_ranking(df)
estocastica = seleccion_estocastica(df)

def fitness_prom(rutas):
    return df[df['Cromosoma'].isin(rutas)]['Fitness'].mean()

resultados = {
    "Ruleta": {"Individuos": ruleta, "Fitness Promedio": fitness_prom(ruleta)},
    "Torneo": {"Individuos": torneo, "Fitness Promedio": fitness_prom(torneo)},
    "Elitismo": {"Individuos": elitismo, "Fitness Promedio": fitness_prom(elitismo)},
    "Ranking": {"Individuos": ranking, "Fitness Promedio": fitness_prom(ranking)},
    "Estocástica": {"Individuos": estocastica, "Fitness Promedio": fitness_prom(estocastica)}
}

# Visualización
plt.figure(figsize=(10,6))
plt.bar(resultados.keys(), [v["Fitness Promedio"] for v in resultados.values()])
plt.title("Comparación de Fitness Promedio por Mecanismo de Selección")
plt.ylabel("Fitness Promedio")
plt.grid(axis='y')
plt.show()


## Paso 4 y 5: Cruce de un punto y mutación simple

In [ ]:

def cruce_un_punto(p1, p2):
    punto = random.randint(1, len(p1)-1)
    return p1[:punto] + p2[punto:], p2[:punto] + p1[punto:]

def mutar(crom, tasa=0.1):
    return ''.join('1' if c == '0' and random.random() < tasa else
                   '0' if c == '1' and random.random() < tasa else c
                   for c in crom)

nuevos = {}
for metodo, datos in resultados.items():
    padres = datos['Individuos']
    if len(padres) >= 2:
        h1, h2 = cruce_un_punto(padres[0], padres[1])
        nuevos[metodo] = [mutar(h1), mutar(h2)]

nuevos



## ✅ Conclusiones Finales

- Los mecanismos de selección como **elitismo** y **torneo** favorecen la mejora rápida pero tienden a reducir diversidad.
- Métodos como **ruleta**, **ranking** y **estocástica universal** permiten mantener la variabilidad en la población.
- La **mutación simple** es fundamental para evitar la convergencia prematura.
- La visualización del fitness promedio permite seleccionar estrategias más eficientes para la evolución genética.
- Este enfoque es útil en decisiones dinámicas como precios de habitaciones, tarifas, recursos o combinaciones de marketing.
